In [117]:
# Using simplegmail abstraction instead of writing gmail logic from scratch
# https://github.com/jeremyephron/simplegmail
from simplegmail import Gmail, query
# ollama
import ollama
# langchain
from langchain_community.llms import Ollama
from langchain import PromptTemplate


In [107]:
# Get gmail service - may need to authenticate - check https://github.com/jeremyephron/simplegmail
gmail = Gmail()

In [124]:
llm = Ollama(model="llama3.1", stop=["<|eot_id|>"]) # Added stop token

In [125]:
def get_model_response(user_prompt, system_prompt):
    # NOTE: No f string and no whitespace in curly braces
    template = """
        <|begin_of_text|>
        <|start_header_id|>system<|end_header_id|>
        {system_prompt}
        <|eot_id|>
        <|start_header_id|>user<|end_header_id|>
        {user_prompt}
        <|eot_id|>
        <|start_header_id|>assistant<|end_header_id|>
        """

    # Added prompt template
    prompt = PromptTemplate(
        input_variables=["system_prompt", "user_prompt"],
        template=template
    )
    
    # invoking the model
    response = llm(prompt.format(system_prompt=system_prompt, user_prompt=user_prompt))
    
    return response

In [110]:
# Get mails from past _ units
numberOf = 10
messages = gmail.get_messages(query=query.construct_query(newer_than=(numberOf, "hour"), labels=[["INBOX"]]))

In [ ]:
print(f'Total messages: {len(messages)}')
for message in messages:
    print(f"""\n --- from: {message.sender} date:{message.date} ---
        subject: {message.subject}""")
    print(message.html)

In [135]:
user_mail_prompt = 'Give a simple Yes or No answer to the question - Do you think the mail I recieved is regarding a SOFTWARE JOB APPLICATION?'

In [136]:
print(f'Total messages: {len(messages)}')
for message in messages:
    print(f"""from: {message.sender} date:{message.date}
        subject: {message.subject}""")
    result = get_model_response(user_prompt=user_mail_prompt, system_prompt=message.subject+message.html)
    print(f' --- {result=} ---')
    # break
    

Total messages: 6
from: Skydive Space Center <skydivesc@outlook.com> date:2024-08-16 13:21:31-04:00
        subject: Re: Skydive appointment reminder - Response required.
 --- result='Yes.' ---
from: 0pk-239-e0n@user.dice.com date:2024-08-16 12:28:31-04:00
        subject: Full time onsite role || PL/SQL Developer || Juno Beach FL (Onsite)
 --- result='No' ---
from: Skydive Space Center <skydivesc@outlook.com> date:2024-08-16 12:17:19-04:00
        subject: Skydive appointment reminder - Response required.
 --- result='No.' ---
from: People First <system@successfactors.com> date:2024-08-16 12:05:27-04:00
        subject: Thank You for Your Interest in Job GOVERNMENT OPERATIONS CONSULTANT III - 65000491
 --- result='No' ---
from: Starbucks Rewards <Starbucks@e.starbucks.com> date:2024-08-16 09:08:32-04:00
        subject: Open on your birthday 🎁 (Actually, you can open now)
 --- result='No.' ---
from: Conservice <ebill@ebill.conservicemail.com> date:2024-08-16 08:14:37-04:00
        sub